In [2]:
import numpy as np
import cv2 ## opencv
import mediapipe as mp

mp_drawing=mp.solutions.drawing_utils ## drawing landmarks
mp_pose=mp.solutions.pose ## importing pose models

In [4]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    # Ensure the angle is between 0 and 180 degrees
    if angle > 180.0:
        angle = 360 - angle
    
    return angle  # Return the angle



# SHOULDER PRESS COUNTER

In [ ]:
# Initialize webcam feed (0 is the default webcam)
cap = cv2.VideoCapture(0)

# Curl counter and current stage (up/down)
counter = 0 
stage = "down"

# Setup Mediapipe Pose instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # Flip horizontally for mirror view

        # Convert frame from BGR to RGB for Mediapipe processing
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False  # Improve performance

        # Run pose estimation
        results = pose.process(image)

        # Convert back to BGR for OpenCV display
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            # Extract pose landmarks
            landmarks = results.pose_landmarks.landmark

            # Get coordinates of left arm
            shoulder_l = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow_l = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist_l = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            hip_l = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            
            # Get coordinates of right arm
            shoulder_r = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow_r = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist_r = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            hip_r = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

        
            angle_shoulder_l = calculate_angle(hip_l, shoulder_l, elbow_l)
            angle_shoulder_r = calculate_angle(hip_r, shoulder_r, elbow_r)
            
            # Calculate elbow angles (to ensure arms are extended)
            angle_elbow_l = calculate_angle(shoulder_l, elbow_l, wrist_l)
            angle_elbow_r = calculate_angle(shoulder_r, elbow_r, wrist_r)


            if angle_shoulder_l > 160 and angle_shoulder_r > 160 and angle_elbow_l > 150 and angle_elbow_r > 150:
                if stage == "down":
                    stage = "up"
                    counter += 1
                    print(f"Count: {counter}")
            
            # Check for "down" position - arms at or below shoulder level
            # For shoulder press, shoulder angle is smaller when arms are down
            elif angle_shoulder_l < 100 and angle_shoulder_r < 100:
                stage = "down"

        except:
            pass  # In case landmarks are not detected

        # Draw a box to show rep count and stage
        cv2.rectangle(image, (0, 0), (225, 80), (245, 117, 16), -1)

        # Display reps count
        cv2.putText(image, 'SHOULDER PRESS', (15, 12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Display current stage (up/down)
        cv2.putText(image, 'STAGE', (80, 12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)


        # Show the processed video feed
        cv2.imshow('Shoulder press counter', image)

        # Exit on pressing 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()


Count: 1
Count: 2
Count: 3
Count: 4
Count: 5
Count: 6
Count: 7
